In [1]:
  # student_performance_importer.py
import pandas as pd
import mysql.connector
from mysql.connector import Error

def main():
    # Configuration - Update these values
    config = {
        'host': 'localhost',
        'user': 'root',
        'password': 'Ytmp3.eu',
        'database': 'student_db',
        'csv_path': 'trained_model_predictions.csv'
    }

    # SQL statements
    sql_commands = {
        'create_db': "CREATE DATABASE IF NOT EXISTS student_performance",
        'use_db': "USE student_performance",
        'create_table': """
            CREATE TABLE IF NOT EXISTS student_records (
                Age INT,
                Year_of_Study INT,
                Attendance DECIMAL(5,2),
                Assignment_Score DECIMAL(5,2),
                Midterm_Score DECIMAL(5,2),
                Final_Score DECIMAL(5,2),
                Tuition_Paid INT,
                Outstanding_Balance INT,
                Books_Borrowed INT,
                Library_Visits INT,
                Days_Absent INT,
                Gender_Male BOOLEAN,
                Department_Computer_Science BOOLEAN,
                Department_Electrical_Engineering BOOLEAN,
                Parents_Education_Primary BOOLEAN,
                Parents_Education_University BOOLEAN,
                Chronic_Illness_Yes BOOLEAN,
                Performance_Pass BOOLEAN
            )
        """,
        'insert_data': """
            INSERT INTO student_records VALUES 
            (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """,
        'verify_data': "SELECT * FROM student_records LIMIT 5"
    }

    def setup_database():
        try:
            # Connect to MySQL server without specifying a database
            connection = mysql.connector.connect(
                host=config['host'],
                user=config['user'],
                password=config['password']
            )
            
            cursor = connection.cursor()
            
            # Create database if not exists
            cursor.execute(sql_commands['create_db'])
            cursor.execute(sql_commands['use_db'])
            
            # Create table
            cursor.execute(sql_commands['create_table'])
            
            print("✅ Database and table created successfully")
            return connection
            
        except Error as e:
            print(f"❌ Database setup error: {e}")
            return None

    def import_data(connection):
        try:
            # Read and prepare CSV data
            df = pd.read_csv(config['csv_path'])
            df.columns = df.columns.str.replace(' ', '_')
            df['Performance_Pass'] = df['Performance_Pass'].apply(
                lambda x: 1 if x == 'pass' else 0
            )
            
            cursor = connection.cursor()
            
            # Insert data row by row
            for _, row in df.iterrows():
                cursor.execute(sql_commands['insert_data'], tuple(row))
            
            connection.commit()
            print(f"✅ Successfully imported {len(df)} records")
            
        except Error as e:
            print(f"❌ Data import error: {e}")
        except FileNotFoundError:
            print(f"❌ CSV file not found at: {config['csv_path']}")

    def verify_data(connection):
        try:
            cursor = connection.cursor(dictionary=True)
            cursor.execute(sql_commands['verify_data'])
            results = cursor.fetchall()
            
            print("\n📋 Sample Data Verification (first 5 records):")
            for row in results:
                print(row)
                
        except Error as e:
            print(f"❌ Verification error: {e}")

    # Execute the complete workflow
    print("🚀 Starting database setup and data import process...")
    
    # Step 1: Database setup
    db_connection = setup_database()
    
    if db_connection:
        # Step 2: Data import
        import_data(db_connection)
        
        # Step 3: Verification
        verify_data(db_connection)
        
        # Cleanup
        db_connection.close()
        print("🔌 Database connection closed")
    
    print("🏁 Process completed")

if __name__ == "__main__":
    main()

🚀 Starting database setup and data import process...
✅ Database and table created successfully
✅ Successfully imported 10 records

📋 Sample Data Verification (first 5 records):
{'Age': 20, 'Year_of_Study': 2, 'Attendance': Decimal('85.50'), 'Assignment_Score': Decimal('75.00'), 'Midterm_Score': Decimal('65.00'), 'Final_Score': Decimal('70.00'), 'Tuition_Paid': 500000, 'Outstanding_Balance': 0, 'Books_Borrowed': 10, 'Library_Visits': 20, 'Days_Absent': 2, 'Gender_Male': 1, 'Department_Computer_Science': 1, 'Department_Electrical_Engineering': 0, 'Parents_Education_Primary': 0, 'Parents_Education_University': 1, 'Chronic_Illness_Yes': 0, 'Performance_Pass': 1}
{'Age': 22, 'Year_of_Study': 3, 'Attendance': Decimal('90.00'), 'Assignment_Score': Decimal('88.50'), 'Midterm_Score': Decimal('92.50'), 'Final_Score': Decimal('85.00'), 'Tuition_Paid': 750000, 'Outstanding_Balance': 250000, 'Books_Borrowed': 5, 'Library_Visits': 15, 'Days_Absent': 10, 'Gender_Male': 0, 'Department_Computer_Science